# Setup

In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from lib.FFNNClassifier import FFNNClassifier

from lib.MLPLib import MLPLIB
from lib.Utils import model_comparison, model_scratch_output


In [2]:
def one_hot_encode(y, num_classes=10):
    y = np.asarray(y, dtype=int)
    one_hot = np.zeros((len(y), num_classes), dtype=int)
    one_hot[np.arange(len(y)), y] = 1
    return one_hot

In [3]:
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

# Type conversion
X = X.astype('float32')
y = y.astype('int')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# One-hot encode labels
y_train_one_hot = one_hot_encode(y_train)
y_test_one_hot = one_hot_encode(y_test)

In [4]:
"""
DEFAULT PARAMETERS
"""

lower_bound=5.39294405e-05
upper_bound=1
mean=5.39294405e-05
std=.44
seed=69
hidden_layer_sizes=[128,64,32]
max_iter=15
init_method="normal"
learning_rate_init=0.01
batch_size=50
activation_mlplib="logistic"
activation_ffnn="sigmoid"
l1=0.1
l2=0.1


# Pengaruh depth (banyak layer) dan width (banyak neuron per layer)

In [5]:
# Configuration for depth variation (keeping width constant)
depth_configs = [
    [10, 10],  # 2 layers
    [10, 10, 10],  # 3 layers
    [10, 10, 10, 10]  # 4 layers
]

# Configuration for width variation (keeping depth constant)
width_configs = [
    [5],  # narrow layer
    [15],  # medium layer
    [30]  # wide layer
]

In [6]:
# Experiment for depth variations (keeping width fixed)
print("Depth Variations Experiment:")
for depth_config in depth_configs:
    print(f"\nTesting depth configuration: {depth_config}")

    # Scikit-learn MLP
    sk_mlp = MLPLIB(
        max_iter=max_iter,
        learning_rate_init=learning_rate_init,
        hidden_layer_sizes=depth_config,
        init_method=init_method,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        activation=activation_mlplib,
        verbose=False,
    )

    # Custom MLP
    custom_mlp = FFNNClassifier(
        max_epoch=max_iter,
        learning_rate=learning_rate_init,
        hidden_layer_sizes=depth_config,
        init_method=init_method,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        verbose=0,
        loss_func="categorical_cross_entropy",
        activation_func=[activation_ffnn] * len(depth_config) + ['softmax'],
    )

    model_comparison(sk_mlp, custom_mlp,X_train_scaled, y_train, y_train_one_hot, X_test_scaled, y_test, y_test_one_hot, is_only_show_accuracy=True)

Depth Variations Experiment:

Testing depth configuration: [10, 10]
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:104: RuntimeWarning: overflow encountered in exp
  elif func == 'sigmoid': return 1.0/(1.0 + np.exp(-x))


Accuracy:
 0.6517142857142857

[From Scratch FFNNClassifier]
Accuracy:
 0.6517142857142857

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing depth configuration: [10, 10, 10]
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:104: RuntimeWarning: overflow encountered in exp
  elif func == 'sigmoid': return 1.0/(1.0 + np.exp(-x))


Accuracy:
 0.4722142857142857

[From Scratch FFNNClassifier]
Accuracy:
 0.4722142857142857

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing depth configuration: [10, 10, 10, 10]
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:104: RuntimeWarning: overflow encountered in exp
  elif func == 'sigmoid': return 1.0/(1.0 + np.exp(-x))


Accuracy:
 0.24035714285714285

[From Scratch FFNNClassifier]
Accuracy:
 0.24035714285714285

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal



In [7]:
print("Width Variations Experiment:")
for width in width_configs:
    print(f"\nTesting width configuration: {width}")

    # Scikit-learn MLP
    sk_mlp = MLPLIB(
        max_iter=max_iter,
        learning_rate_init=learning_rate_init,
        hidden_layer_sizes=width,
        init_method=init_method,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        activation=activation_mlplib,
        verbose=False,
    )

    # Custom MLP
    custom_mlp = FFNNClassifier(
        max_epoch=max_iter,
        learning_rate=learning_rate_init,
        hidden_layer_sizes=width,
        init_method=init_method,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        verbose=0,
        loss_func="categorical_cross_entropy",
        activation_func=[activation_ffnn] * len(width) + ['softmax'],
    )

    model_comparison(sk_mlp, custom_mlp,X_train_scaled, y_train, y_train_one_hot, X_test_scaled, y_test, y_test_one_hot, is_only_show_accuracy=True)

Width Variations Experiment:

Testing width configuration: [5]
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:104: RuntimeWarning: overflow encountered in exp
  elif func == 'sigmoid': return 1.0/(1.0 + np.exp(-x))


Accuracy:
 0.7432142857142857

[From Scratch FFNNClassifier]
Accuracy:
 0.7432142857142857

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing width configuration: [15]
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:104: RuntimeWarning: overflow encountered in exp
  elif func == 'sigmoid': return 1.0/(1.0 + np.exp(-x))


Accuracy:
 0.8022142857142858

[From Scratch FFNNClassifier]
Accuracy:
 0.8022142857142858

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing width configuration: [30]
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:104: RuntimeWarning: overflow encountered in exp
  elif func == 'sigmoid': return 1.0/(1.0 + np.exp(-x))


Accuracy:
 0.8432857142857143

[From Scratch FFNNClassifier]
Accuracy:
 0.8432857142857143

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal



# Pengaruh fungsi aktivasi hidden layer

In [8]:
activation_configs = [
    ('linear','identity'),
    ('relu','relu'),
    ('sigmoid','logistic'),
    ('tanh','tanh')
]

bonus_activation_configs = [
    'softsign',
    'softplus'
]

In [9]:
print("Activation Function Variations Experiment:")
for act_custom, act_sklearn in activation_configs:
    print(f"\nTesting activation configuration: {act_custom}")

    # Scikit-learn MLP
    sk_mlp = MLPLIB(
        max_iter=max_iter,
        learning_rate_init=learning_rate_init,
        hidden_layer_sizes=hidden_layer_sizes,
        init_method=init_method,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        activation=act_sklearn,
        verbose=False,
    )

    # Custom MLP
    custom_mlp = FFNNClassifier(
        max_epoch=max_iter,
        learning_rate=learning_rate_init,
        hidden_layer_sizes=hidden_layer_sizes,
        init_method=init_method,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        verbose=0,
        loss_func="categorical_cross_entropy",
        activation_func=[act_custom] * len(hidden_layer_sizes) + ['softmax'],
    )

    model_comparison(sk_mlp, custom_mlp,X_train_scaled, y_train, y_train_one_hot, X_test_scaled, y_test, y_test_one_hot, is_only_show_accuracy=True)

Activation Function Variations Experiment:

Testing activation configuration: linear
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.8857857142857143

[From Scratch FFNNClassifier]
Accuracy:
 0.8857857142857143

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing activation configuration: relu
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.7580714285714286

[From Scratch FFNNClassifier]
Accuracy:
 0.7580714285714286

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing activation configuration: sigmoid
[SKLearn MLPClassifier]


...

Accuracy:
 0.8449285714285715

[From Scratch FFNNClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:104: RuntimeWarning: overflow encountered in exp
  elif func == 'sigmoid': return 1.0/(1.0 + np.exp(-x))


Accuracy:
 0.8449285714285715

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing activation configuration: tanh
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.8095

[From Scratch FFNNClassifier]
Accuracy:
 0.8095

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal



In [10]:
for act_custom in bonus_activation_configs:
    print(f"\nTesting activation configuration: {act_custom}")

    # Custom MLP
    custom_mlp = FFNNClassifier(
        max_epoch=max_iter,
        learning_rate=learning_rate_init,
        hidden_layer_sizes=hidden_layer_sizes,
        init_method=init_method,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        verbose=0,
        loss_func="categorical_cross_entropy",
        activation_func=[act_custom] * len(hidden_layer_sizes) + ['softmax'],
    )

    model_scratch_output(custom_mlp, X_train_scaled, y_train_one_hot, X_test_scaled, y_test_one_hot)


Testing activation configuration: softsign
[From Scratch FFNNClassifier]
Weights:
 [array([[ 0.34849682,  0.20366047,  0.14421766, ..., -0.11011714,
         0.85692686, -0.20459601],
       [ 0.43855953,  0.09489525,  0.03395932, ..., -0.07869318,
         0.03085322, -0.7965541 ],
       [-0.20512688,  0.14743526, -0.8297421 , ..., -0.05077973,
        -0.49387592,  0.17818075],
       ...,
       [-0.6334432 , -0.34678155,  0.25855687, ..., -0.48119783,
        -0.21261859, -0.14547339],
       [ 0.5134772 , -0.6923477 ,  0.23591197, ...,  0.42926645,
         0.39576954, -0.33404246],
       [ 0.28817222, -0.24174216, -0.67546964, ..., -0.4206143 ,
        -0.25590882, -0.3167226 ]], shape=(784, 128), dtype=float32), array([[ 0.26231116,  0.22627188,  0.23337834, ...,  0.32387826,
        -0.35762906,  0.15279643],
       [-0.30226976,  0.03051095,  0.4826649 , ..., -0.04104296,
        -0.5140538 ,  0.8041288 ],
       [-1.0443975 ,  0.4134605 ,  0.00576117, ...,  0.5646582 ,
   

C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:112: RuntimeWarning: overflow encountered in exp
  return np.log(1 + np.exp(x))
C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:283: RuntimeWarning: invalid value encountered in matmul
  a_k = b_k + (h_k_min_1 @ w_k) # numpy will automatically broadcast b_k (row will be copied to match the result from dot) so that this is addable
C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:107: RuntimeWarning: invalid value encountered in subtract
  exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:137: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x)) # sigmoid(x)


Weights:
 [array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]],
      shape=(784, 128), dtype=float32), array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]],
      shape=(128, 64), dtype=float32), array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], shape=(64, 32), dtype=float32), array([[nan, nan, nan, nan, nan, nan, nan, nan,

# Pengaruh learning rate

In [11]:
learning_rates = [0.1, 0.06, 0.006, 0.0009]

In [12]:
print("Learning Rate Variations Experiment:")
for rate in learning_rates:
    print(f"\nTesting activation configuration: {rate}")

    # Scikit-learn MLP
    sk_mlp = MLPLIB(
        max_iter=max_iter,
        learning_rate_init=learning_rate_init,
        hidden_layer_sizes=hidden_layer_sizes,
        init_method=init_method,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        activation=activation_mlplib,
        verbose=False,
    )

    # Custom MLP
    custom_mlp = FFNNClassifier(
        max_epoch=max_iter,
        learning_rate=learning_rate_init,
        hidden_layer_sizes=hidden_layer_sizes,
        init_method=init_method,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        verbose=0,
        loss_func="categorical_cross_entropy",
        activation_func=[activation_ffnn] * len(hidden_layer_sizes) + ['softmax'],
    )

    model_comparison(sk_mlp, custom_mlp, X_train_scaled, y_train, y_train_one_hot, X_test_scaled, y_test, y_test_one_hot, is_only_show_accuracy=True)

Learning Rate Variations Experiment:

Testing activation configuration: 0.1
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:104: RuntimeWarning: overflow encountered in exp
  elif func == 'sigmoid': return 1.0/(1.0 + np.exp(-x))


Accuracy:
 0.8362857142857143

[From Scratch FFNNClassifier]
Accuracy:
 0.8449285714285715

[Comparison Result]
[[-1.2339124e-05  6.1707797e-06 -2.0272801e-06 ...  1.4391768e-05
   3.2699811e-01  1.6529350e-05]
 [ 2.8024482e-02  1.6078342e-05 -6.2238705e-06 ...  1.0345961e-05
  -1.2563138e-05 -2.8385484e-01]
 [ 1.5986505e-05  8.2681636e-06 -3.0757144e-01 ... -1.1510952e-05
  -6.7554668e-02  9.9791687e-06]
 ...
 [-1.6729239e-01  5.4763477e-06  4.2207848e-06 ... -5.8494825e-02
   1.8479319e-05 -1.5473250e-05]
 [ 8.1562303e-02 -2.0938627e-01  7.4062227e-06 ...  2.1383431e-02
   2.8339946e-06 -1.0776417e-05]
 [-1.7302587e-05  7.2310340e-06 -1.9732551e-01 ... -1.5200627e-02
  -1.6433622e-05  1.6390066e-05]] != [[ 0.34849682  0.20366047  0.14421766 ... -0.11011714  0.85692686
  -0.20459601]
 [ 0.43855953  0.09489525  0.03395932 ... -0.07869318  0.03085322
  -0.7965541 ]
 [-0.20512688  0.14743526 -0.8297421  ... -0.05077973 -0.49387592
   0.17818075]
 ...
 [-0.6334432  -0.34678155  0.25855687

C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.8362857142857143

[From Scratch FFNNClassifier]


...

Accuracy:
 0.8362857142857143

[From Scratch FFNNClassifier]
Accuracy:
 0.8449285714285715

[Comparison Result]
[[-1.2339124e-05  6.1707797e-06 -2.0272801e-06 ...  1.4391768e-05
   3.2699811e-01  1.6529350e-05]
 [ 2.8024482e-02  1.6078342e-05 -6.2238705e-06 ...  1.0345961e-05
  -1.2563138e-05 -2.8385484e-01]
 [ 1.5986505e-05  8.2681636e-06 -3.0757144e-01 ... -1.1510952e-05
  -6.7554668e-02  9.9791687e-06]
 ...
 [-1.6729239e-01  5.4763477e-06  4.2207848e-06 ... -5.8494825e-02
   1.8479319e-05 -1.5473250e-05]
 [ 8.1562303e-02 -2.0938627e-01  7.4062227e-06 ...  2.1383431e-02
   2.8339946e-06 -1.0776417e-05]
 [-1.7302587e-05  7.2310340e-06 -1.9732551e-01 ... -1.5200627e-02
  -1.6433622e-05  1.6390066e-05]] != [[ 0.34849682  0.20366047  0.14421766 ... -0.11011714  0.85692686
  -0.20459601]
 [ 0.43855953  0.09489525  0.03395932 ... -0.07869318  0.03085322
  -0.7965541 ]
 [-0.20512688  0.14743526 -0.8297421  ... -0.05077973 -0.49387592
   0.17818075]
 ...
 [-0.6334432  -0.34678155  0.25855687

C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.8362857142857143

[From Scratch FFNNClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:104: RuntimeWarning: overflow encountered in exp
  elif func == 'sigmoid': return 1.0/(1.0 + np.exp(-x))


Accuracy:
 0.8449285714285715

[Comparison Result]
[[-1.2339124e-05  6.1707797e-06 -2.0272801e-06 ...  1.4391768e-05
   3.2699811e-01  1.6529350e-05]
 [ 2.8024482e-02  1.6078342e-05 -6.2238705e-06 ...  1.0345961e-05
  -1.2563138e-05 -2.8385484e-01]
 [ 1.5986505e-05  8.2681636e-06 -3.0757144e-01 ... -1.1510952e-05
  -6.7554668e-02  9.9791687e-06]
 ...
 [-1.6729239e-01  5.4763477e-06  4.2207848e-06 ... -5.8494825e-02
   1.8479319e-05 -1.5473250e-05]
 [ 8.1562303e-02 -2.0938627e-01  7.4062227e-06 ...  2.1383431e-02
   2.8339946e-06 -1.0776417e-05]
 [-1.7302587e-05  7.2310340e-06 -1.9732551e-01 ... -1.5200627e-02
  -1.6433622e-05  1.6390066e-05]] != [[ 0.34849682  0.20366047  0.14421766 ... -0.11011714  0.85692686
  -0.20459601]
 [ 0.43855953  0.09489525  0.03395932 ... -0.07869318  0.03085322
  -0.7965541 ]
 [-0.20512688  0.14743526 -0.8297421  ... -0.05077973 -0.49387592
   0.17818075]
 ...
 [-0.6334432  -0.34678155  0.25855687 ... -0.48119783 -0.21261859
  -0.14547339]
 [ 0.5134772  -0.

# Perbandingan Weight

In [13]:
weight_configs =  ['normal', 'zero', 'uniform']
additional_weight_configs = ["xavier_normal","xavier_uniform","he_normal","he_uniform"]

In [14]:
print("Weight Variations Experiment:")
for weight_config in weight_configs:
    print(f"\nTesting weight configuration: {weight_config}")

    # Scikit-learn MLP
    sk_mlp = MLPLIB(
        max_iter=max_iter,
        learning_rate_init=learning_rate_init,
        hidden_layer_sizes=hidden_layer_sizes,
        init_method=weight_config,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        activation=activation_mlplib,
        verbose=False,
    )

    # Custom MLP
    custom_mlp = FFNNClassifier(
        max_epoch=max_iter,
        learning_rate=learning_rate_init,
        hidden_layer_sizes=hidden_layer_sizes,
        init_method=weight_config,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        verbose=0,
        loss_func="categorical_cross_entropy",
        activation_func=[activation_ffnn] * len(hidden_layer_sizes) + ['softmax'],
    )


    model_comparison(sk_mlp, custom_mlp, X_train_scaled, y_train, y_train_one_hot, X_test_scaled, y_test, y_test_one_hot, is_only_show_accuracy=True)

Weight Variations Experiment:

Testing weight configuration: normal
[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:104: RuntimeWarning: overflow encountered in exp
  elif func == 'sigmoid': return 1.0/(1.0 + np.exp(-x))


Accuracy:
 0.8449285714285715

[From Scratch FFNNClassifier]
Accuracy:
 0.8449285714285715

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal


Testing weight configuration: zero
[SKLearn MLPClassifier]
Accuracy:
 0.11428571428571428

[From Scratch FFNNClassifier]
Accuracy:
 0.11428571428571428

[Comparison Result]
[[2.2941954e-06 2.2941954e-06 2.2941954e-06 ... 2.2941954e-06
  2.2941954e-06 2.2941954e-06]
 [2.2941954e-06 2.2941954e-06 2.2941954e-06 ... 2.2941954e-06
  2.2941954e-06 2.2941954e-06]
 [2.2941954e-06 2.2941954e-06 2.2941954e-06 ... 2.2941954e-06
  2.2941954e-06 2.2941954e-06]
 ...
 [2.2941954e-06 2.2941954e-06 2.2941954e-06 ... 2.2941954e-06
  2.2941954e-06 2.2941954e-06]
 [2.2941954e-06 2.2941954e-06 2.2941954e-06 ... 2.2941954e-06
  2.2941954e-06 2.2941954e-06]
 [2.2941954e-06 2.2941954e-06 2.2941954e-06 ... 2.2941954e-06
  2.2941954e-06 2.2941954e-06]] != [[3.6016156e-06 3.60

C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:104: RuntimeWarning: overflow encountered in exp
  elif func == 'sigmoid': return 1.0/(1.0 + np.exp(-x))


Accuracy:
 0.11428571428571428

[Comparison Result]
✅ Weight is equal
✅ Bias is equal
✅ Prediction is equal
✅ Prediction Probability is equal
✅ Loss is equal
✅ Accuracy is equal



In [15]:
print("Weight Variations Experiment:")
for weight_config in additional_weight_configs:
    print(f"\nTesting weight configuration: {weight_config}")

    # Custom MLP
    custom_mlp = FFNNClassifier(
        max_epoch=max_iter,
        learning_rate=learning_rate_init,
        hidden_layer_sizes=hidden_layer_sizes,
        init_method=weight_config,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed,
        batch_size=batch_size,
        verbose=0,
        loss_func="categorical_cross_entropy",
        activation_func=[activation_ffnn] * len(hidden_layer_sizes) + ['softmax'],
    )


    model_scratch_output(custom_mlp, X_train_scaled, y_train_one_hot, X_test_scaled, y_test_one_hot)

Weight Variations Experiment:

Testing weight configuration: xavier_normal
[From Scratch FFNNClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:104: RuntimeWarning: overflow encountered in exp
  elif func == 'sigmoid': return 1.0/(1.0 + np.exp(-x))


Weights:
 [array([[ 0.03708485,  0.02166989,  0.01534337, ..., -0.01172553,
         0.09119716, -0.02178094],
       [ 0.04667024,  0.01009398,  0.00360856, ..., -0.00838107,
         0.00327797, -0.08478315],
       [-0.02183744,  0.01568583, -0.08831536, ..., -0.00541024,
        -0.05256906,  0.01895807],
       ...,
       [-0.06742322, -0.03691377,  0.02751252, ..., -0.05121972,
        -0.02263478, -0.0154885 ],
       [ 0.05464375, -0.07369245,  0.02510242, ...,  0.04568118,
         0.04211609, -0.03555795],
       [ 0.03066449, -0.02573441, -0.07189611, ..., -0.04477181,
        -0.02724217, -0.03371459]], shape=(784, 128), dtype=float32), array([[ 0.07391854,  0.09762227,  0.0952058 , ...,  0.12700953,
        -0.18754998,  0.11732751],
       [-0.14079145, -0.02479143,  0.10170957, ..., -0.10474101,
        -0.08239639,  0.2188448 ],
       [-0.3020406 ,  0.08188853, -0.00455378, ...,  0.08547567,
         0.03392957,  0.10718075],
       ...,
       [ 0.02982106, -0.119625

C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:104: RuntimeWarning: overflow encountered in exp
  elif func == 'sigmoid': return 1.0/(1.0 + np.exp(-x))


Weights:
 [array([[ 0.03999778,  0.02337201,  0.01654856, ..., -0.01264654,
         0.0983605 , -0.02349178],
       [ 0.05033609,  0.01088684,  0.003892  , ..., -0.00903939,
         0.00353545, -0.09144268],
       [-0.02355272,  0.01691791, -0.09525234, ..., -0.0058352 ,
        -0.05669824,  0.02044719],
       ...,
       [-0.07271917, -0.03981327,  0.02967357, ..., -0.05524292,
        -0.02441269, -0.01670509],
       [ 0.05893589, -0.07948083,  0.02707416, ...,  0.04926933,
         0.04542422, -0.03835095],
       [ 0.03307312, -0.02775579, -0.07754339, ..., -0.04828853,
        -0.02938198, -0.0363628 ]], shape=(784, 128), dtype=float32), array([[ 0.08942035,  0.13168676,  0.10267941, ...,  0.16195412,
        -0.23958616,  0.10902213],
       [-0.12479687, -0.00569725,  0.10673644, ..., -0.08415622,
        -0.14352779,  0.27333504],
       [-0.36243618,  0.09824809, -0.01299943, ...,  0.10597096,
         0.02758318,  0.10509517],
       ...,
       [ 0.03119531, -0.146565

# Pengaruh Regularisasi

In [ ]:
# Scikit-learn MLP
sk_mlp = MLPLIB(
    max_iter=max_iter,
    learning_rate_init=learning_rate_init,
    hidden_layer_sizes=hidden_layer_sizes,
    init_method=init_method,
    lower_bound=lower_bound,
    upper_bound=upper_bound,
    mean=mean,
    std=std,
    seed=seed,
    batch_size=batch_size,
    activation=activation_mlplib,
    verbose=False,
)

# Custom MLP
custom_mlp = FFNNClassifier(
    max_epoch=max_iter,
    learning_rate=learning_rate_init,
    hidden_layer_sizes=hidden_layer_sizes,
    init_method=init_method,
    lower_bound=lower_bound,
    upper_bound=upper_bound,
    mean=mean,
    std=std,
    seed=seed,
    batch_size=batch_size,
    verbose=0,
    loss_func="categorical_cross_entropy",
    activation_func=[activation_ffnn] * len(hidden_layer_sizes) + ['softmax'],
)


model_comparison(sk_mlp, custom_mlp, X_train_scaled, y_train, y_train_one_hot, X_test_scaled, y_test, y_test_one_hot, is_only_show_accuracy=True)

[SKLearn MLPClassifier]


In [16]:
# Scikit-learn MLP
sk_mlp = MLPLIB(
    max_iter=max_iter,
    learning_rate_init=learning_rate_init,
    hidden_layer_sizes=hidden_layer_sizes,
    init_method=init_method,
    lower_bound=lower_bound,
    upper_bound=upper_bound,
    mean=mean,
    std=std,
    seed=seed,
    batch_size=batch_size,
    activation=activation_mlplib,
    verbose=False,
    alpha_l1=l1
)

# Custom MLP
custom_mlp = FFNNClassifier(
    max_epoch=max_iter,
    learning_rate=learning_rate_init,
    hidden_layer_sizes=hidden_layer_sizes,
    init_method=init_method,
    lower_bound=lower_bound,
    upper_bound=upper_bound,
    mean=mean,
    std=std,
    seed=seed,
    batch_size=batch_size,
    verbose=0,
    loss_func="categorical_cross_entropy",
    activation_func=[activation_ffnn] * len(hidden_layer_sizes) + ['softmax'],
    l1 = l1
)


model_comparison(sk_mlp, custom_mlp, X_train_scaled, y_train, y_train_one_hot, X_test_scaled, y_test, y_test_one_hot, is_only_show_accuracy=True)

[SKLearn MLPClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy:
 0.8455

[From Scratch FFNNClassifier]


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:104: RuntimeWarning: overflow encountered in exp
  elif func == 'sigmoid': return 1.0/(1.0 + np.exp(-x))


Accuracy:
 0.8455

[Comparison Result]
[[ 1.25334943e-02 -2.04976459e-06 -1.27172225e-05 ... -2.26038901e-06
   5.20470560e-01 -1.36140570e-05]
 [ 1.02603912e-01  1.83600187e-05 -3.02603439e-07 ... -1.41504988e-05
   1.37276784e-05 -4.60157186e-01]
 [ 1.54448335e-05  5.31605838e-06 -4.93295789e-01 ... -1.91982090e-05
  -1.57920301e-01 -5.13424675e-06]
 ...
 [-2.97288984e-01 -1.08179813e-02 -1.83952561e-05 ... -1.45242214e-01
   2.75011917e-06 -3.18426100e-06]
 [ 1.77501559e-01 -3.56105864e-01  1.37034640e-05 ...  9.33108330e-02
   5.98134138e-02  2.83952613e-06]
 [ 8.59687134e-07 -4.67437712e-06 -3.39252919e-01 ... -8.46586823e-02
  -1.31968245e-05  4.98523514e-06]] != [[ 1.25334943e-02 -2.04976459e-06 -1.27172225e-05 ... -2.26038901e-06
   5.20470560e-01 -1.36140570e-05]
 [ 1.02603912e-01  1.83600187e-05 -3.02603439e-07 ... -1.41504988e-05
   1.37276784e-05 -4.60157186e-01]
 [ 1.54448335e-05  5.31605838e-06 -4.93295789e-01 ... -1.91982090e-05
  -1.57920301e-01 -5.13424675e-06]
 ...
 [

In [ ]:
# Scikit-learn MLP
sk_mlp = MLPLIB(
    max_iter=max_iter,
    learning_rate_init=learning_rate_init,
    hidden_layer_sizes=hidden_layer_sizes,
    init_method=init_method,
    lower_bound=lower_bound,
    upper_bound=upper_bound,
    mean=mean,
    std=std,
    seed=seed,
    batch_size=batch_size,
    activation=activation_mlplib,
    verbose=False,
    alpha=l2
)

# Custom MLP
custom_mlp = FFNNClassifier(
    max_epoch=max_iter,
    learning_rate=learning_rate_init,
    hidden_layer_sizes=hidden_layer_sizes,
    init_method=init_method,
    lower_bound=lower_bound,
    upper_bound=upper_bound,
    mean=mean,
    std=std,
    seed=seed,
    batch_size=batch_size,
    verbose=0,
    loss_func="categorical_cross_entropy",
    activation_func=[activation_ffnn] * len(hidden_layer_sizes) + ['softmax'],
    l2 = l2
)


model_comparison(sk_mlp, custom_mlp, X_train_scaled, y_train, y_train_one_hot, X_test_scaled, y_test, y_test_one_hot, is_only_show_accuracy=True)